# Demand prediction baseline solution

Отчет:
1. По данным о продажах за 2012-2014 годы предсказать продажи определенных товаров в 2015 году.
2. В ходе изучения данных было установлено, что столбцы 'f[i]' и 'f[i+30]' идентичны для каждой строки исходных данных. В связи с этим выбрасываем половину из них. Более того, обнаружена утечка данных в тестовой выборке: изучение данных показало зависимость между продажами и одной из фич.

Однако с помощью утечки можно спрогнозировать только часть результатов тестовой выборки, оставшуюся часть прогнозируем, обучаясь на тестовой выборке с использованием подходов, указанных ниже.
3. Методы:

    1) Использование метрики качества MAE. В связи с тем, что оценка качества производится с помощью $SMAPE = \frac{200%}{n}\sum_{n=1}^{N}\frac{|y - predict|}{|y| + |predict|}$, было принято решение минимизировать MAE. Это дало некоторое преимущество и улучшило результат.
    
    2) Выбор недель. Тестовые данные относятся только к 3-5 неделям 2015 года. Посколько речь идет о продажах, которые могут зависеть от сезона, возникло предположение, что стоит обучать модель только на той части тренировочной выборки, которая относится к 3-5 неделям. Улучшило результат.
    
    3) Кодирование item_id. Попытка заменить item_id на средние значения тех или иных фич только ухудшила прогноз. Нормировка и взаимодействие со значениями item_id внутри столбца не принесло никаких результатов.
    
    4) Изменение параметров модели. Увеличивая n_estimators и их максимальную глубину, получили несколько переобученную модель, дающую при этом хорошие результаты. Хорошо увеличило качество.
    
    5) Работа со сдвигом. Попытка обучения различных моделей для каждого из сдвигов дала результат несколько хуже, чем приведенная ниже модель.
    
    6) Используем утечку данных. Получаем переобученную на данную утечку модель, которая, тем не менее, дает очень хороший результат на тестовой выборке.
4. Код ниже
5. Приоценке качества использовалась функция SMAPE

In [327]:
import pandas as pd

test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")
train = pd.read_csv("train.tsv")
for i in range(31, 61):
    st = 'f' + str(i)
    train = train.drop([st], axis = 1)
    test = test.drop([st], axis = 1)

In [328]:
import numpy as np
import math

fifth_week_nums = []
res = []
m = min(test['item_id'])
M = max(test['item_id'])
test['item_id'] -= m

for i in range(0, len(test['item_id'])):
    if (test['week'][i] != 5):
        item_id = test['item_id'][i]
        week = test['week'][i]
        shift = test['shift'][i]
        num = test['Num'][i]
        for j in range(0, len(test['item_id'])):
                if (test['item_id'][j] == item_id):
                    aim_week = test['week'][j] - test['shift'][j]
                    if (week == aim_week):
                        res.append([num, math.ceil(test['f30'][j] * 1.609999)])
                        break
    else:
        fifth_week_nums.append(test['Num'][i])

In [329]:
print len(res)
frac = 1 # fraction of learning examples used for model fitting
    
#sample the train set if your don't want to deel with all examples
X = train.sample(frac=frac, random_state=42)
#X = X[X['week'].isin([5])]
y = X['y'].copy()
X = X.drop(['Num','y'], axis=1)

1335


In [330]:
def SMAPE(pred, y):
        return float(200) / len(y) * sum(abs(pred - y) / (abs(y) + abs(pred)))

In [331]:
import xgboost as xgb

model = xgb.XGBRegressor(n_estimators=800, max_depth=20, learning_rate=0.05)
model.fit(X, y, eval_metric="mae")

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=20,
       min_child_weight=1, missing=None, n_estimators=800, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [332]:
preds = model.predict(test[test['week'] == 5].drop(['Num'], axis = 1))
for i in range(0, len(preds)):
    res.append([fifth_week_nums[i], math.ceil(preds[i])])

In [333]:
ind = []
nums = []
y = []
mean = 0
for i in range(0, 2016):
    nums.append(test['Num'][i])
for i in range(0, len(res)):
    ind.append(res[i][0])
    mean += res[i][1]
r = list(set(nums) - set(ind))
print r
mean /= len(res)
for i in r:
    res.append([i, mean])

[350619, 351037, 350572, 351018, 351019, 349900, 349901, 350573, 350866, 350591, 349748, 350590, 351065, 351064, 350420, 349946, 349947, 351036, 350618, 349918, 349919]


In [334]:
res.sort()
y = []
n = []
for i in range(0, len(res)):
    y.append(res[i][1])
    n.append(res[i][0])

In [335]:
# In GBM you can get some negative predictions:

sample_submission['Num']= n
sample_submission['y'] = y

sample_submission

,Num,y
0,348622,1498.0
1,348623,27034.0
2,348624,297561.0
3,348625,29472.0
4,348626,17.0
5,348627,146243.0
6,348628,67736.0
7,348629,111942.0
8,348630,102631.0
9,348631,3542.0


In [336]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [337]:
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)